<a href="https://colab.research.google.com/github/tsvoronos/API202-students/blob/main/hw/API203_hw1_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API-203M A PROBLEM SET #1

---


**Due on Wednesday, March 8, at 11:59 p.m.**

**I - INSTRUCTIONS**  
To successfully complete this problem set, please follow these steps:

1. **Create a copy of this Jupyter notebook in your own drive by clicking `Copy to Drive` in the menubar (this is explained below in more detail below) - *if you do not do this your work will not be saved!***
    1. Remember to save your work frequently by pressing `command-S` or clicking `File > Save` in the menubar.
    1. We recommend completing this problem set in Google Chrome.
2. **Insert all your answers into your copy of the document.** 
    1. Please include every portion of your submission in this document unless a separate electronic file is explicitly requested. 
    3. All numerical calculations should be done in the notebook itself, using R code. If you have to do calculations by hand, include a picture of your handwritten work.
    2. To add an image of handwritten work, screenshots, or anything else, add a new text cell and click the small image icon in the menubar above the cell. After inserting an image, press the `esc` key to show the image. If you have trouble with this method, upload your image to [Freeimage.host](https://freeimage.host), copy a Markdown link, and paste it in a text cell.
3. **Once your document is complete, please save and submit the notebook on Canvas as a PDF.** 
    1. Click `View > Expand sections` in the menubar to make sure all work is visible (if it is greyed out then it is already visible).
    1. Click `Runtime > Run all` in the menubar to make sure all of your code is executed.
    1. Click `File > Print` in the menubar and choose the `Save as PDF` option in Chrome to export your notebook as a PDF. 
    1. Check to make sure answers do not get cut off - when you go to print the PDF we recommend clicking on `More settings` and setting `Margins` to `None`.
    1. Alternatively, use [this Google Chrome extension](https://chrome.google.com/webstore/detail/gofullpage-full-page-scre/fdpohaocaechififmbbbbbknoalclacl) to create a single PDF without having to worry about margins.
    1. Submit your file on Canvas.


**II - IDENTIFICATION**
1. **Your Full Name:** `     `  

2. **Group Members (classmates with whom you worked on this problem set):**  
    1. `     `
    2. `     `
    3. `     `
    4. `     `
    
3. **Compliance with HKS Academic Code**  
We abide by the Harvard Kennedy School Academic code for all aspects of the course. In terms of problem sets, unless explicitly written otherwise, the norms are the following: You are free (and encouraged) to discuss problem sets with your classmates. However, you must hand in your own unique written work and code in all cases. Any copy/paste of another’s work is plagiarism. In other words, you may work with your classmate(s), sitting side-by-side (physically or remotely!) and going through the problem set question by question, but you must each type your own answers and your own code. For more details, please see syllabus.

    **I certify that my work in this problem set complies with the HKS Academic Code**
    - [ ] Yes
    - [ ] No

---

## Load `R` libraries and data

**Please refer to Sheet 1 in this [R Cheat Sheet](https://bit.ly/HKS-R) which includes the commands you learned last semester in addition to a number of additional ones.**

The code cell below imports the R tidyverse. Make sure to run it before starting the problem set!

*Note: Click the "play" button that appears when you hover over a cell to run it. The first time you do this you may receive an alert that this notebook was not authored by Google. If so, click "Run anyway" to proceed.*

In [1]:
library(tidyverse)

check_installed <- require(fixest)
if(check_installed==F){
  install.packages("fixest")
  require(fixest)
}

# Load data
seatbelts <- read_csv("https://raw.githubusercontent.com/tsvoronos/API202-students/main/data/seatbelts.csv")
head(seatbelts)


Loading required package: fixest

Rows: 765 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): state
dbl (13): year, fips, vmt, fatalityrate, sb_useage, speed65, speed70, drinka...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


state,year,fips,vmt,fatalityrate,sb_useage,speed65,speed70,drinkage21,ba08,income,age,primary,secondary
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AK,1983,2,3358,0.04466945,NA,0,0,1,0,17973,28.23497,0,0
AK,1984,2,3589,0.03733630,NA,0,0,1,0,18093,28.34354,0,0
AK,1985,2,3840,0.03307291,NA,0,0,1,0,18925,28.37282,0,0
AK,1986,2,4008,0.02519960,NA,0,0,1,0,18466,28.39665,0,0
AK,1987,2,3900,0.01948718,NA,0,0,1,0,18021,28.45325,0,0
AK,1988,2,3841,0.02525384,NA,0,0,1,0,18447,28.85142,0,0


# PART 1: Policy Application - Fixed Effects

Traffic crashes are the leading cause of death for Americans between the ages of 5 and 32. Through various spending policies, the federal government has encouraged states to institute mandatory seat belt laws to reduce the number of fatalities and serious injuries. In this exercise you will investigate how effective these laws are in increasing seat belt use and reducing fatalities. The data set analyzed contains a panel of data from 50 U.S. states, plus the District of Columbia, for the years 1983-1997. 

This dataset was provided by Professor Liran Einav of Stanford University and was used in his paper with Alma Cohen, "The Effects of Mandatory Seat Belt Laws on Driving Behavior and Traffic Fatalities" in The Review of Economics and Statistics, 2003, Vol. 85, No. 4, pp 828-843.  The variables are as follows:

* `fatalityrate`: number of fatalities per million of traffic miles
* `sb_useage`: seat belt usage rate (measured as a proportion between 0 and 1 )
* `speed65`: binary variable for 65 mph speed limit
* `speed70`: binary variable for 70 mph or higher speed limit
* `ba08`: binary variable for blood alcohol limit $\leq .08 \%$
* `drinkage21`: binary variable for age 21 drinking age
* `income`: per capita income
* `age`: mean age
* `primary`: binary variable for primary enforcement of seat belt laws
* `secondary`: binary variable for secondary enforcement of seat belt laws
* `vmt`: millions of traffic miles per year (note: total fatalities = fatalityrate $x$ vmt)
* `state`: state
* `year`: year


**1. Run the following three regressions:**

Hint: Use the `feols` command from the `fixest` library (which we have been using for robust standard errors) to specify fixed effects. The syntax is:

```
feols(y ~ x1 + x2 | fe1 + fe2, data = dataset)
```

Where $y$ is the dependent variable, $x 1$ and $x 2$ are independent variables, and $f e 1$ and $f e 2$ are the variables that we wish to include as fixed effects.

**(a) Regression A: Fatality rate per million traffic miles on the seat belt usage rate, controlling for:**

* whether the state has a 65 mph speed limit,
* whether the state has a 70 mph or higher speed limit,
* whether the state has a $0.08 \%$ blood alcohol limit,
* whether the state has a 21 drinking age,
* the **log** of per capita income,
* mean age, and 
* mean age squared

In [2]:
# Your code here



**(b) Regression B: add state fixed effects to regression A**

In [3]:
# Your code here


**(c) Regression C: add year fixed effects to regression B**

In [4]:
# Your code here




Run the script below installing the package modelsummary (this may take a minute or so to install).

In [5]:
check_installed <- require(modelsummary)
if(check_installed==F){
  install.packages("modelsummary")
  require(modelsummary)
}

Loading required package: modelsummary

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘modelsummary’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘webshot’, ‘bayestestR’, ‘datawizard’, ‘checkmate’, ‘insight’, ‘kableExtra’, ‘parameters’, ‘performance’, ‘tables’


Loading required package: modelsummary



**(d) Run all three regressions in parts (a)-(c), and report the coefficient estimate and the standard error on the `sb_useage` variable in a nicely formatted table.**

Hint: remember, you learnt how to create a nicely formatted table in API202.

In [6]:
# Your code here





**2. What does regression A suggest about the effect of increased seat belt use on fatalities? Interpret the `sb_useage` variable.**

Your answer here:



**3. Interpret sb_useage in regression B.  What does regression B suggest about the effect of increased seat belt use on fatalities? In two to three sentences, offer one possible explanation for the differences between regression A and B and explain which model you believe to be more credible and why.**



Your answer:




**4. If the seatbelt usage rates within each of the 50 states was exactly constant during this time period, what would change in regression B?**

Your answer here: 


**5. What does regression C suggest about the effect of increased seat belt use on fatalities? What does the inclusion of year fixed-effects help control?**


Your answer here:



**6. If the only change in seatbelt usage rates was a 5 point increase in all states in 1990 due to a national safety campaign, what would change in regression C?**

Your answer here:


**7. Compare the economic significance of the coefficient on sb_useage in regressions A, B, and C by calculating the predicted change in the number of traffic fatalities if seat belt use increased from $60 \%$ to $80 \%$ for a hypothetical state where vehicle miles traveled is equivalent to the sample mean, all else equal?**

(Hint: total fatalities $=$ fatality rate $*$ vmt. Use the average vmt to calculate the predicted change in total number of traffic fatalities for each regression)

Your answer here:


In [7]:
# Code if you calculate the magnitudes of seatbelt usage in R


# PART II: Relative Risk and HIV



This portion of the problem set is based on the 2011 Dupas paper that you read about in the Kenya Intervention section of the Youth Impact Case:

  > Dupas, Pascaline. (2011). Do Teenagers Respond to HIV Risk Information? Evidence from a Field Experiment in Kenya. American Economic Journal: Applied Economics, 3(1), 1-34.

In the paper, Dupas ran a randomized experiment to test the effect of different kinds of information on the sexual behavior of teenagers in Kenya. 323 primary schools were randomized into receiving one of the following interventions, both of the interventions, or neither of them:

* Treatment 1: "Relative Risk" information campaign providing information on the distribution of HIV in the population by gender and age group. This was the treatment of interest.
* Treatment 2: "Teacher Training" program involving the official Kenya HIV/AIDS curriculum, including estimates on HIV/AIDS prevalence in the region. This was the "standard" treatment in Kenya at the time.

All interventions were allocated at the school level, and targeted students in Grade 8 of each treated school.

In her primary specification, Dupas ran the following regression as a Linear Probability Model (note the equation is shown over multiple lines for ease of reading):

Equation 1
$$
\begin{aligned}
\text { childbearing }_{i s}=\beta_1 & +\beta_2 T T_s+\beta_3 R R_s \\
& +\gamma_1 I_{1 i}+\cdots+\gamma_j I_{j i} \\
& +\eta_1 S_{1 s}+\cdots+\eta_k S_{k s}+\varepsilon_{i s}
\end{aligned}
$$

Where:
* $\text {childbearing }_{i s}$ is whether student $i$ in school $s$ has started childbearing
* $T T_s$: $=1$ if the student's school received the Teacher training HIV/AIDS curriculum, $=0$ otherwise
* $R R_S$: $=1$ if the student's school received the Relative Risk information campaign, $=0$ otherwise
* $I_{1 i} \ldots I_{j i}$ is a set of $j$ individual-level controls (and $\gamma_1 \ldots \gamma_j$ their corresponding coefficients)
* $S_{1 s} \ldots S_{k s}$ is a set of $k$ school-level controls (and $\eta_1 \ldots \eta_k$ their corresponding coefficients)

For this next part of the problem set, you will conduct an analysis similar to column (1) of Table 3 in the Dupas paper (page 19). Note that your results will differ from the authors' to an extent - this is OK! This is because the authors also included geographic variables on district and division, which we will not do because of complications this creates in our coding.

The article specifies "Individual characteristics" and "Primary school characteristics" for these regressions. **Again, do not include the variables for district and division.** The following table describes which variables are included in the Dupas_DID.dta dataset:


* Treatment Variables

  * `sampleSD`: RR information
  * `HIVtreat`: TT on HIV/AIDS curriculum


* Individual Characteristics

  * `age`: Age
  * `agemissing`: Whether the student is missing a value for their age
  * `double8`: Whether student is repeating grade 8 at baseline
  * `cohort`: Whether a student was in Grade 8 or not

* Primary School Characteristics

  * `sdkcpe`: School 2003 KCPE score (a national exam for 8th graders) standardized by district
  * `missingKCPE`: Missing value for KCPE score
  * `clsize`: Class size
  * `girl8perboy8`: Girl/Boy gender ratio
  * `G_promorate`: Fraction of Girls promoted from 7 to 8 at end of 2003
  * `teacherperpupil`: Teacher-pupil ratio as of 2003
  * `timegroup`: Grouping of surveys into two groups (balanced across treatment/control groups)
  * `d05v1`: Date of 1st school visit in 2005
  * `d05v2`: Date of 2nd school visit in 2005
  * `total_2km`: Number of other primary schools within 2km radius sampled for study
  * `schoolid`: An identifier for the student’s school


In [8]:
library(tidyverse)

# Load data
dupas_did <- read_csv("https://raw.githubusercontent.com/tsvoronos/API202-students/main/data/dupas_did.csv")

Rows: 11972 Columns: 118
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (8): sex, std02, married04v1, married04v2, district, division, zone, g...
dbl (108): pupilid, yrbirth, baseline, sch03v1, std03v1, pres03v1, d03v1, ag...
lgl   (2): new2005v1, new05v2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


**1. First, restrict your dataset to students for whom the variable cohort = 1 (this will be explained later).**

**Estimate a linear probability model on the outcome `fertafter12`, a dummy variable which equals 1 if the student was pregnant one year after the intervention, and 0 otherwise. Use all the control variables listed above.**

Note: your R output from this regression should correspond closely to column (1) of Table 3. However, there will be some differences in the standard error as in this exercise we do not cluster the standard errors at the school level.

In [9]:
# Your code here:



**2. Would it be possible to add school fixed effects to this regression? Why or why not?**

Your answer here:

**3. In addition, Dupas ran a difference in differences specification. Using the same schools as before, she compared students in Grade 8 to students in either Grade 7 or Grade 9 in both Relative Risk and nonRelative Risk schools. In Relative Risk schools, only 8thgraders received the intervention, while 7th and 9th graders did not. Note that these "control" grades (7 or 9 ) were not included in Equation 1. This specification was in the following form:**

Equation 2
$$
\begin{aligned}
\text { childbearing }_{i g s}=\alpha_1 & +\alpha_2 T T_s+\alpha_3 R R_s+\alpha_4 \text { Grade } 8_g+\alpha_5 R R_s * \text { Grade } 8_g \\
& +\delta_1 I_{1 i}+\cdots+\delta_j I_{j i} \\
& +\theta_1 S_{1 s}+\cdots+\theta_k S_{k s}+\omega_{i g s}
\end{aligned}
$$

Where:
* $\text {childbearing}_{i g s}$ is whether student $i$ in Grade $g$ in school $s$ has started childbearing
* Grade $8_g$:$=1$ if the student was in Grade $8=0$ otherwise
* All other variables are the same as in the previous regression

Note: You do not need to have read tthe paper in depth to answer these questions. All needed information is in the text of this problem set!

**(a) Which coefficient(s) represents the difference-in-differences estimate for the impact of the Relative Risk intervention? Carefully interpret that quantity in the context of the regression (you can refer to "individual controls" and "school controls" rather than naming each control variable).**

Your answer here:

**(b) The variable for whether a student was in Grade 8 or not in this dataset is called `cohort`. Recreate the difference-in-differences regression in equation 2 by running a regression in R.**

In [10]:
# Your code here:



**(c) Describe a potential omitted variable or source of error that is controlled for in this second regression that is not accounted for in the primary regression. Be specific to this context.**

Your answer here:



**(d) Explain the assumption required for this difference-in-differences specification to yield the correct estimates in this example. Note that this might be challenging since there are no time "trends" in this model; instead of a pre and post period, we're comparing two different grades at the same time** 

Hint: remember that the goal of the difference-in-differences design is to make the comparison group an approximation of the treatment's counterfactual via differences or changes, rather than absolute levels.

Your answer here:


**4. In addition to *Equation 2*, Dupas ran an additional specification that included school fixed effects. In this specification, which variables from *Equation 2* could not be included in the regression? You can experiment with the data to see if your intuition is correct, using the `schoolid` variable and the fixed effects code from Part 1 of this problem set.**

Note: if you do experiment with the data, note that the `sampleSD` variable will get dropped as it becomes collinear with school fixed effects in the regression. This variable is no longer our primary variable of interest, so it's OK that it gets subsumed by the fixed effects!:

Your answer here:



**5. A policymaker who is smart but not well-versed in statistics does not understand what "school fixed effects" means. In plain language (i.e., not using words like "variation," "omitted variables," or "bias"), explain to this policymaker what comparisons are being made to estimate the impact of the program when using school fixed effects, and why this might be useful to do [one short paragraph].**

Your answer here:

